In [ ]:
import sys
import rospy
import rospkg
import moveit_commander
import tf
import os 
import numpy as np 
import copy
import time

from geometry_msgs.msg import (
    PoseStamped,
    Pose,
    Point,
    Quaternion,
)
from gazebo_msgs.srv import (SpawnModel,DeleteModel)
from std_msgs.msg import Header
from geometry_msgs.msg import (PoseStamped,Pose,Point,Quaternion)
from trajectory_msgs.msg import JointTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint
from structure.class_robot import ROBOT
from structure.class_structure import *
from structure.utils.util_ik import * 

import rospy 
import numpy as np 
import math 

print("Packages Loaded.")
rospy.init_node("object_spawn")

### Set Moveit 

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)
robot         = moveit_commander.RobotCommander()
scene         = moveit_commander.PlanningSceneInterface()
group_name    = robot.get_group_names()
print("Moveit control group name:%s"%group_name)
arm_group     = moveit_commander.MoveGroupCommander("arm")
gripper_group = moveit_commander.MoveGroupCommander("gripper")

## Spawn-Delete Object 

In [ ]:
def spawn_gazebo_model(model_path, model_name, model_pose, reference_frame="world"):
  """
  Spawn model in gazebo
  """
  model_xml = ''
  with open(model_path, "r") as model_file:
    model_xml = model_file.read().replace('\n', '')
  rospy.wait_for_service('/gazebo/spawn_sdf_model')
  try:
    spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model', SpawnModel)
    resp_sdf= spawn_sdf(model_name, model_xml, "/", model_pose, reference_frame)
  except rospy.ServiceException:
    rospy.logerr("Spawn SDF service call failed")

def delete_gazebo_model(models):
  """
  Delete model in gazebo
  """
  try:
    delete_model = rospy.ServiceProxy('/gazebo/delete_model', DeleteModel)
    for a_model in models:
      resp_delete = delete_model(a_model)
  except rospy.ServiceException:
    rospy.loginfo("Delete Model service call failed")


### Set Parameters 

In [ ]:
# Parameters 
table_height    = 0.79
# Cylinder 
cylinder_height = 0.2
cylinder_radius = 0.03
cylinder_pos_x  = 0.99 
cylinder_pos_y  = 0 
cylinder_pos_z  = table_height+cylinder_height/2.
# Box 
box_height      = 0.06
box_x           = 0.06 
box_y           = 0.06
box_pos_x       = 0.99 
box_pos_y       = 0 
box_pos_z       = table_height+box_height/2.

In [ ]:
""" RViz """
# Cylinder 
cylinder_name = "cylinder"
cylinder_pos  = PoseStamped()
cylinder_pos.header.frame_id    = robot.get_planning_frame()
cylinder_pos.pose.position.x    = cylinder_pos_x
cylinder_pos.pose.position.y    = cylinder_pos_y
cylinder_pos.pose.position.z    = cylinder_pos_z
cylinder_pos.pose.orientation.w = 1.0  
# Box 
box_name = "box"
box_pos  = PoseStamped()
box_pos.header.frame_id    = robot.get_planning_frame()
box_pos.pose.position.x    = box_pos_x
box_pos.pose.position.y    = box_pos_y
box_pos.pose.position.z    = box_pos_z
box_pos.pose.orientation.w = 1.0  

""" GAZEBO """
rospack = rospkg.RosPack()
pack_path = rospack.get_path('ir_gazebo')
object_path = pack_path+os.sep+'urdf'+os.sep+'object'+os.sep+'cylinder.sdf'
object_name = cylinder_name
object_pose = Pose(position=Point(x=cylinder_pos_x , y=cylinder_pos_y, z=cylinder_pos_z))


## Trajectory 

In [ ]:


def move_gripper(joint):
    global gripper_joint_names, gripper_pub
    """ Open- Close Gripper """
    gripper        = JointTrajectory()
    gripper_value  = JointTrajectoryPoint()
    gripper.header = Header()
    gripper.joint_names = gripper_joint_names
    gripper_value.positions = [joint] # Open pose
    gripper_value.time_from_start = rospy.Duration.from_sec(0.01)
    gripper.points.append(gripper_value)
    gripper_pub.publish(gripper)

def single_move(joint_seq): 
    Flag = True 
    global arm_joint_names, rate, arm_pub
    """ Move to pick an object """
    arm = JointTrajectory()
    arm_value = JointTrajectoryPoint()
    arm.header = Header()
    arm.joint_names= arm_joint_names
    arm_value.positions       = joint_seq
    arm_value.time_from_start = rospy.Duration.from_sec(0.01)
    arm.points.append(arm_value)
    arm_pub.publish(arm)


def move_arm(joint_seq): 
    Flag = True 
    global arm_joint_names, rate, arm_pub
    while Flag:
        """ Move to pick an object """
        for idx, joints in enumerate(joint_seq):
            joint = joints.reshape([6,])
            arm = JointTrajectory()
            arm_value = JointTrajectoryPoint()
            arm.header = Header()
            arm.joint_names= arm_joint_names
            arm_value.positions       = joint
            arm_value.time_from_start = rospy.Duration.from_sec(0.01)
            arm.points.append(arm_value)
            arm_pub.publish(arm)
            rate.sleep()    
            if idx+1 == len(joint_seq): 
                Flag = False 



In [ ]:
chain = CHAIN(_file_name="structure/utils/ur5e_onrobot.urdf", verbose=False)
chain.add_joi_to_robot()
chain.add_link_to_robot()
chain.fk_chain(1)


target_x = 1
target_y = -0.4

offser_from_base_x = target_x-0.18 #np.linalg.norm(np.array([0.8,0.2])-np.array([0.18,0]))
offser_from_base_y = target_y-0.1135

angle = math.atan2(offser_from_base_y, offser_from_base_x, )
print("angle",angle)
start_x = 0.75#dist *math.cos(angle)
start_y = 0#math.sin(angle) 
print(start_x, start_y)

tcp_pos_start = [start_x, start_y, 0.86]
wrist_pos_start = [start_x-0.26, start_y, 0.86]
print(tcp_pos_start, wrist_pos_start)



variable = make_ik_input(target_name = ['wrist_3_joint', 'gripper_tcp_joint'],
                            target_pos  = [[0,0,0], tcp_pos_start],
                            target_rot  = [[0, 3.14, angle-1.57],[0,0,0]],
                            solve_pos   = [0, 1],
                            solve_rot   = [1, 0],
                            weight_pos  = 1,
                            weight_rot  = 1,
                            disabled_joi_id = [0],
                            joi_ctrl_num=7) # Including base joint
q_list = chain.get_q_from_ik(variable)
contro_q_list_start = q_list[1:7] # Excluding base joint
contro_q_list_start = contro_q_list_start.reshape(-1,1)

contro_q_list_start[0] = angle
contro_q_list_start[4] = 1.57

print(chain.joint[6].name, chain.joint[6].p)
print(chain.joint[8].name, chain.joint[8].p)
print(chain.joint[9].name, chain.joint[9].p)
print(chain.joint[10].name, chain.joint[10].p)

tcp_pos = [target_x, target_y, 0.86]
wrist_pos = [target_x-0.26, target_y, 0.86]



variable = make_ik_input(target_name = ['wrist_3_joint', 'gripper_tcp_joint'],
                            target_pos  = [[0,0,0], tcp_pos],
                            target_rot  = [[0, 3.14, angle-1.57],[0,0,0]],
                            solve_pos   = [0, 1],
                            solve_rot   = [1, 0],
                            weight_pos  = 1,
                            weight_rot  = 1,
                            disabled_joi_id = [0],
                            joi_ctrl_num=7) # Including base joint
q_list = chain.get_q_from_ik(variable)
control_q_list = q_list[1:7] # Excluding base joint
contro_q_list = control_q_list.reshape(1,-1)

print(tcp_pos, wrist_pos)
print(chain.joint[6].name, chain.joint[6].p)
print(chain.joint[8].name, chain.joint[8].p)
print(chain.joint[9].name, chain.joint[9].p)
print(chain.joint[10].name, chain.joint[10].p)


In [ ]:

cylinder_name = "cylinder"
cylinder_pos  = PoseStamped()
cylinder_pos.header.frame_id    = robot.get_planning_frame()
cylinder_pos.pose.position.x    = target_x
cylinder_pos.pose.position.y    = target_y
cylinder_pos.pose.position.z    = cylinder_pos_z
cylinder_pos.pose.orientation.w = 1.0  

arm_joint_names = ['shoulder_pan_joint','shoulder_lift_joint','elbow_joint', 
                'wrist_1_joint', 'wrist_2_joint','wrist_3_joint']
gripper_joint_names = ['gripper_finger1_joint']
rate    = rospy.Rate(10)
arm_pub = rospy.Publisher('/arm_controller/command', JointTrajectory, queue_size=10)
gripper_pub = rospy.Publisher('/gripper_controller/command', JointTrajectory, queue_size=10)

touch_links = ["gripper_finger1_finger_link", "gripper_finger2_finger_link"]


In [ ]:


single_move(control_q_list)
# Add a cylinder on RViz 
scene.add_cylinder(name=object_name, height=cylinder_height, radius=cylinder_radius, pose=cylinder_pos)
# Add a cylinder on GAZEBO 


### Plan

In [ ]:
init_joints = [0.9, -0.6596, 1.3364, 0.0350, 0, 0]
single_move(init_joints)

delete_gazebo_model([object_name])



In [ ]:
spawn_gazebo_model(object_path, object_name, object_pose)


In [ ]:


# single_move(contro_q_list_start)
move_gripper(0.8)


In [ ]:
# Move to start pose 
# direction_joints = [angle, -0.97181214,  2.32397468, -1.35171505,  1.13762996, -0.0087976 ]
# direction_joints = [0,0,0,0,0,0]

# control_q_list[0] = angle
# control_q_list[4] = 1.57
single_move(contro_q_list_start)



In [ ]:
# total_q_list = np.linspace(contro_q_list_start, control_q_list, 20)
# move_arm(total_q_list)

In [24]:
def linaer_move(target_pos, desired_vel, offset_angle):
    start_pos = [0.77, 0, 0.82] 
    """ Start Pose """
    variable_start = make_ik_input(target_name = ['wrist_3_joint', 'gripper_tcp_joint'],
                                target_pos  = [[0,0,0], start_pos],
                                target_rot  = [[0, 3.14, offset_angle-1.57],[0,0,0]],
                                solve_pos   = [0, 1],
                                solve_rot   = [1, 0],
                                weight_pos  = 1,
                                weight_rot  = 1,
                                disabled_joi_id = [0],
                                joi_ctrl_num=7) # Including base joint        
    q_list_start = chain.get_q_from_ik(variable_start)
    q_list_start = q_list_start[1:7] # Excluding base joint
    ctrl_q_list_start = q_list_start.reshape(-1,)        
    ctrl_q_list_start[0] = offset_angle
    ctrl_q_list_start[4] = 1.57
    """ Goal Pose """
    variable = make_ik_input(target_name = ['wrist_3_joint', 'gripper_tcp_joint'],
                                target_pos  = [[0,0,0], target_pos],
                                target_rot  = [[0, 3.14, offset_angle-1.57],[0,0,0]],
                                solve_pos   = [0, 1],
                                solve_rot   = [1, 0],
                                weight_pos  = 1,
                                weight_rot  = 1,
                                disabled_joi_id = [0],
                                joi_ctrl_num=7) # Including base joint
    q_list_goal = chain.get_q_from_ik(variable)
    q_list_goal = q_list_goal[1:7] # Excluding base joint
    ctrl_q_list_goal = q_list_goal.reshape(-1,)        
    desired_time = get_desired_time(start_pos, target_pos, desired_vel)
    interpoled_q = np.linspace(start=ctrl_q_list_start, stop=ctrl_q_list_goal, num=int(10*desired_time))
    return interpoled_q

In [25]:
inter = linaer_move(tcp_pos, 0.2, angle)

In [26]:
print(inter.shape)
print(inter)

(23, 6)
[[-5.59475787e-01 -9.31358632e-01  2.19821974e+00 -1.26686111e+00
   1.57000000e+00 -1.59254767e-03]
 [-5.62158397e-01 -9.18537168e-01  2.15924356e+00 -1.24070639e+00
   1.56731739e+00 -1.59255214e-03]
 [-5.64841007e-01 -9.05715704e-01  2.12026738e+00 -1.21455168e+00
   1.56463478e+00 -1.59255662e-03]
 [-5.67523617e-01 -8.92894239e-01  2.08129121e+00 -1.18839696e+00
   1.56195217e+00 -1.59256109e-03]
 [-5.70206226e-01 -8.80072775e-01  2.04231503e+00 -1.16224225e+00
   1.55926956e+00 -1.59256556e-03]
 [-5.72888836e-01 -8.67251311e-01  2.00333885e+00 -1.13608753e+00
   1.55658695e+00 -1.59257004e-03]
 [-5.75571446e-01 -8.54429846e-01  1.96436267e+00 -1.10993282e+00
   1.55390434e+00 -1.59257451e-03]
 [-5.78254056e-01 -8.41608382e-01  1.92538649e+00 -1.08377811e+00
   1.55122173e+00 -1.59257898e-03]
 [-5.80936665e-01 -8.28786917e-01  1.88641031e+00 -1.05762339e+00
   1.54853912e+00 -1.59258346e-03]
 [-5.83619275e-01 -8.15965453e-01  1.84743413e+00 -1.03146868e+00
   1.54585651e+00

In [27]:
move_arm(inter)